In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
from numpy import load
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.8 MB/s eta 0:00:00


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation/ILDC_expert/anno_dataset.csv')

In [4]:
import pickle
file_path = '/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation_caselawbert/only_annotation/attention_anno_full/att_wt_of_caselawbert_bigru_anno_full.pkl'
with open(file_path, 'rb') as file:
    att_scores_anno = pickle.load(file)

In [5]:
output_dir = '/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation_caselawbert/CaseLawBERT_bigru_occ/saved_model_multi_caselawbert'
tokenizer = AutoTokenizer.from_pretrained(output_dir)

In [6]:
def caselawbert_detok(tokens):
    # Initialize an empty list for the detokenized sequence
    detokenized_sequence = []

    # Initialize a variable for handling '##' signs
    current_token = ""

    # Iterate through the tokens in the sequence
    for token in tokens:
        if token.startswith("##"):
            # If the token starts with '##', remove '##' and append the rest of the token to the previous token
            current_token += token[2:]
        else:
            # If the token does not start with '##', check if it is a punctuation mark
            if current_token and token in string.punctuation:
                # If the current token is a punctuation mark, attach it to the previous token
                current_token += token
            else:
                # If the current token is not a punctuation mark, add the previous token (if present) to the detokenized sequence
                if current_token:
                    detokenized_sequence.append(current_token)
                # Add the current token to the detokenized sequence
                current_token = token

    # Make sure to add the last token to the detokenized sequence
    if current_token:
        detokenized_sequence.append(current_token)


    tokens_concat = []

    prev_tok=""

    for token in detokenized_sequence:
      if token.endswith('-'):
          prev_tok = token
      else:
          if (prev_tok != ""):
              token = prev_tok + token
              prev_tok = ""
          tokens_concat.append(token)

    if (prev_tok!=""):
        tokens_concat.append(prev_tok)

    # Join the tokens in the detokenized sequence into a string
    detokenized_text = " ".join(tokens_concat)

    return detokenized_text


In [7]:
max_indexes = []
for i in range(len(att_scores_anno)):
  index_max= att_scores_anno[i][0].argmax()
  max_indexes.append(index_max)

In [8]:
final_explanations_att = {}
for i in range(len(df)):
  index = df.index[i]
  all_toks = tokenizer.tokenize(df.iloc[i]['text'])
  if(len(all_toks) > 10000):
      all_toks = all_toks[len(all_toks)-10000:]
  splitted_toks = []
  total_tokens = len(all_toks)
  chunk_size = 510
  overlap_size = 100

  end = total_tokens
  if (end<101):
    splitted_toks.insert(0, all_toks)
  else:
    while end > 100:
      start = max(0, end - chunk_size)
      chunk = all_toks[start:end]
      splitted_toks.insert(0, chunk)
      if start!=0:
        end = start + overlap_size
      else:
        end=0

  c= splitted_toks[max_indexes[i]]
  new_c = ["." if token == "[UNK]" else token for token in c]
  explanation_of_this_doc = caselawbert_detok(new_c)
  doc_name = df.iloc[i]['name']
  final_explanations_att[doc_name] = explanation_of_this_doc

In [9]:
final_explanations_att

{'1960_12.txt': 'a rani for her enjoyment in order to maintain her status and dignity showed that the grant of the village was not by way of maintenance but merely to maintain a tradition of the family for keeping up the status and dignity of the appellant. and this. it was submitted. was fortified by the letter of the appellants husband dated november 19. 1949. where his highness stated that the appellants abru. prestige. would go if she were to lose the village. reference in this connection was also made to the appellants letter dated may 26. 1949. wherein she insisted that the village should not be taken away from her as this would be unreasonable. arbitrary and contrary to the spirit of the covenant. in our opinion the connection of the appellant is well founded. the tribunal has found after going into all the documents including the document containing the words to maintain her status and dignity and the letter of the appellants husband to mr. buch which mentioned the word abru th

In [11]:
import json
json_file = open("/content/drive/MyDrive/Thesis/Models_whole_data/case_explanation_caselawbert/only_annotation/attention_anno_full/att_exp_anno_full.json", "w")
json.dump(final_explanations_att, json_file)